In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

[]


In [3]:
!git clone https://www.github.com/emanhamed/Houses-dataset

Cloning into 'Houses-dataset'...
remote: Enumerating objects: 2161, done.
remote: Total 2161 (delta 0), reused 0 (delta 0), pack-reused 2161
Receiving objects: 100% (2161/2161), 176.26 MiB | 20.54 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [4]:
import keras
import cv2
from sklearn.preprocessing import LabelBinarizer,StandardScaler,MinMaxScaler
from keras.layers import Dense,Activation,Conv2D,BatchNormalization,Dropout,MaxPool2D,Flatten
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [5]:
cd Houses-dataset/'Houses Dataset'

/kaggle/working/Houses-dataset/Houses Dataset


In [6]:
cols = ['Bedroom','Bathroom','Area','Zip','Price']
df = pd.read_csv('HousesInfo.txt',header = None,sep = ' ',names = cols)
cat = ['Bathroom','Bedroom','Area']
mm = MinMaxScaler()
lb = LabelBinarizer()
zips = df['Zip'].value_counts().keys().tolist()
occ = df['Zip'].value_counts().tolist()
zip_data = pd.DataFrame()
zip_data['Zip'] = zips
zip_data['occ'] = occ
keep_ind = []
del_ind = []
for i in range(len(df)):
    zc = df['Zip'][i]
    id = zip_data[zip_data['Zip']==zc].index.tolist()
    if zip_data['occ'][id[0]]>20:
        keep_ind.append(i)
    else:
        del_ind.append(i)
df.drop(del_ind,inplace = True)
df[cat] = mm.fit_transform(df[cat])
images = []
for i in range(len(df)):
    id = df.index[i]

    path1 = str(id+1)+'_frontal.jpg'
    i1 = cv2.imread(path1)
    i1 = cv2.resize(i1,(128,128))
    i1 = cv2.cvtColor(i1,cv2.COLOR_BGR2RGB)
    path2 = str(id+1)+'_bathroom.jpg'
    i2 = cv2.imread(path2)
    i2 = cv2.resize(i2,(128,128))
    i2 = cv2.cvtColor(i2,cv2.COLOR_BGR2RGB)

    path3 = str(id+1)+'_bedroom.jpg'
    i3 = cv2.imread(path3)
    i3 = cv2.resize(i3,(128,128))
    i3 = cv2.cvtColor(i3,cv2.COLOR_BGR2RGB)

    path4 = str(id+1)+'_kitchen.jpg'
    i4 = cv2.imread(path4)
    i4 = cv2.resize(i4,(128,128))
    i4 = cv2.cvtColor(i4,cv2.COLOR_BGR2RGB)

    i = np.zeros((256,256,3),dtype = 'uint8')
    i[0:128,0:128] = i1
    i[128:256,128:256] = i2
    i[0:128,128:256] = i3
    i[128:256,0:128] = i4

    images.append(i)
images = np.array(images)
train_df,test_df,train_images,test_images = train_test_split(df,images,test_size = 0.2,random_state = 10)
train_zip2 = lb.fit_transform(train_df['Zip'])
test_zip2 = lb.fit_transform(test_df['Zip'])
train_y2 = train_df['Price']
test_y2 = test_df['Price']
mxm = train_y2.max()
train_y2 = train_y2/mxm
test_y2 = test_y2/mxm
train_df.drop(columns = ['Zip','Price'],inplace = True)
test_df.drop(columns = ['Zip','Price'],inplace = True)
train_x2 = np.hstack([train_df,train_zip2])
test_x2 = np.hstack([test_df,test_zip2])
train_y2 = np.array(train_y2)
train_y2 = train_y2.reshape((-1,1))
test_y2 = np.array(test_y2)
test_y2 = test_y2.reshape((-1,1))

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [12]:
def nn(op):
    model = keras.models.Sequential()
    model.add(Dense(16,input_dim = 11,activation = 'relu'))
    model.add(Dense(8,activation = 'relu'))
    model.add(Dense(4,activation = 'relu'))
    if op:
        model.add(Dense(1,activation = 'linear'))
    return model

In [13]:
def cnn(op):
    model = keras.models.Sequential()
    
    ip = keras.models.Input((256,256,3))
    
    x = Conv2D(16,kernel_size=(3,3),strides=(1,1),padding = 'same')(ip)
    x = Activation('relu')(x)
    x = BatchNormalization(axis = -1)(x)
    x = MaxPool2D(pool_size = (2,2))(x)
    
    x = Conv2D(32,kernel_size=(3,3),strides=(1,1),padding = 'same')(x)
    x = Activation('relu')(x)
    x = BatchNormalization(axis = -1)(x)
    x = MaxPool2D(pool_size = (2,2))(x)
    
    x = Conv2D(64,kernel_size=(3,3),strides=(1,1),padding = 'same')(x)
    x = Activation('relu')(x)
    x = BatchNormalization(axis = -1)(x)
    x = MaxPool2D(pool_size = (2,2))(x)
    
    x = Conv2D(128,kernel_size=(3,3),strides=(1,1),padding = 'same')(x)
    x = Activation('relu')(x)
    x = BatchNormalization(axis = -1)(x)
    x = MaxPool2D(pool_size = (2,2))(x)
    
    x = Flatten()(x)
    
    x = Dense(32,activation='relu')(x)
    x = BatchNormalization(axis = -1)(x)
    x = Dropout(0.35)(x)
    
    x = Dense(16,activation='relu')(x)
    x = BatchNormalization(axis = -1)(x)
    x = Dropout(0.35)(x)
    
    x = Dense(8,activation='relu')(x)
    x = BatchNormalization(axis = -1)(x)
    x = Dropout(0.4)(x)
    
    x = Dense(4,activation='relu')(x)
    
    if op:
        x = Dense(1,activation='linear')(x)
        
    model = keras.models.Model(inputs = ip,outputs = x)
    
    return model

In [10]:
opt = keras.optimizers.adam(lr = 0.001,decay = 0.00000555555)

In [14]:
nn_model = nn(op = False)
cnn_model = cnn(op = False)

In [15]:
ip_cat = m=keras.layers.concatenate([nn_model.output,cnn_model.output])

In [16]:
X = Dense(4,activation = 'relu')(ip_cat)
X = Dense(1,activation='linear')(X)

In [17]:
final_model = keras.models.Model(inputs = [nn_model.input,cnn_model.input],outputs = X)

In [19]:
final_model.compile(optimizer = opt,loss = 'mean_absolute_percentage_error')

In [20]:
final_model.fit([train_x2,train_images],train_y2,validation_data=([test_x2,test_images],test_y2),epochs = 150,batch_size = 8)

Train on 307 samples, validate on 77 samples
Epoch 1/150
307/307 [==============================] - 7s 22ms/step - loss: 395.7104 - val_loss: 169.4365
Epoch 2/150
307/307 [==============================] - 1s 3ms/step - loss: 205.8722 - val_loss: 70.4998
Epoch 3/150
307/307 [==============================] - 1s 3ms/step - loss: 94.6030 - val_loss: 67.1504
Epoch 4/150
307/307 [==============================] - 1s 3ms/step - loss: 92.9179 - val_loss: 69.0988
Epoch 5/150
307/307 [==============================] - 1s 3ms/step - loss: 73.3769 - val_loss: 73.1968
Epoch 6/150
307/307 [==============================] - 1s 3ms/step - loss: 80.4037 - val_loss: 69.0572
Epoch 7/150
307/307 [==============================] - 1s 3ms/step - loss: 74.4992 - val_loss: 68.3801
Epoch 8/150
307/307 [==============================] - 1s 3ms/step - loss: 75.7792 - val_loss: 67.7635
Epoch 9/150
307/307 [==============================] - 1s 3ms/step - loss: 68.2794 - val_loss: 67.0617
Epoch 10/150
307/307 [==

307/307 [==============================] - 1s 3ms/step - loss: 25.8765 - val_loss: 24.5407
Epoch 80/150
307/307 [==============================] - 1s 3ms/step - loss: 28.0972 - val_loss: 21.9389
Epoch 81/150
307/307 [==============================] - 1s 3ms/step - loss: 25.1711 - val_loss: 21.9773
Epoch 82/150
307/307 [==============================] - 1s 3ms/step - loss: 25.6791 - val_loss: 30.0288
Epoch 83/150
307/307 [==============================] - 1s 3ms/step - loss: 26.2833 - val_loss: 20.9828
Epoch 84/150
307/307 [==============================] - 1s 3ms/step - loss: 25.4547 - val_loss: 21.0577
Epoch 85/150
307/307 [==============================] - 1s 3ms/step - loss: 26.9792 - val_loss: 25.1277
Epoch 86/150
307/307 [==============================] - 1s 3ms/step - loss: 24.6035 - val_loss: 24.1679
Epoch 87/150
307/307 [==============================] - 1s 3ms/step - loss: 25.1934 - val_loss: 21.5666
Epoch 88/150
307/307 [==============================] - 1s 3ms/step - loss: 2

In [23]:
final_pred = final_model.predict([test_x2,test_images])

In [24]:
np.mean(np.abs(((test_y2-final_pred)/test_y2)*100))

20.995285543776347

In [30]:
cd ..

/kaggle/working/Houses-dataset


In [31]:
cd ..

/kaggle/working


In [32]:
rm -rf Houses-dataset

In [28]:
final_model.save_weights('final.h5')

100_bathroom.jpg  221_bathroom.jpg  342_bathroom.jpg  463_bathroom.jpg
100_bedroom.jpg   221_bedroom.jpg   342_bedroom.jpg   463_bedroom.jpg
100_frontal.jpg   221_frontal.jpg   342_frontal.jpg   463_frontal.jpg
100_kitchen.jpg   221_kitchen.jpg   342_kitchen.jpg   463_kitchen.jpg
101_bathroom.jpg  222_bathroom.jpg  343_bathroom.jpg  464_bathroom.jpg
101_bedroom.jpg   222_bedroom.jpg   343_bedroom.jpg   464_bedroom.jpg
101_frontal.jpg   222_frontal.jpg   343_frontal.jpg   464_frontal.jpg
101_kitchen.jpg   222_kitchen.jpg   343_kitchen.jpg   464_kitchen.jpg
102_bathroom.jpg  223_bathroom.jpg  344_bathroom.jpg  465_bathroom.jpg
102_bedroom.jpg   223_bedroom.jpg   344_bedroom.jpg   465_bedroom.jpg
102_frontal.jpg   223_frontal.jpg   344_frontal.jpg   465_frontal.jpg
102_kitchen.jpg   223_kitchen.jpg   344_kitchen.jpg   465_kitchen.jpg
103_bathroom.jpg  224_bathroom.jpg  345_bathroom.jpg  466_bathroom.jpg
103_bedroom.jpg   224_bedroom.jpg   345_bedroom.jpg   466_bedroom.jpg
103_frontal.jpg 